In [ ]:
# To generate the history file `emob_history.jsonl` run the following command:
# uv run extract_git_history.py -i data\ch.bfe.ladestellen-elektromobilitaet-status.json -o emob_history.jsonl --start-at 7d398e70718a3f519230aa35ee4dc6a75faf5be2

In [2]:
import pandas as pd
import json
import jsonlines

In [3]:
df = pd.read_json('emob_history.jsonl', lines=True)

In [11]:
records_status = []
with jsonlines.open("emob_history.jsonl", "r") as reader:
    for data in reader.iter(type=dict, skip_invalid=True):
        timestamp = data["_git_committed_datetime"]
        for operator in data.get("EVSEStatuses", []):
            operator_id = operator.get("OperatorID", "")
            operator_name = operator.get("OperatorName", "")

            for evse in operator.get("EVSEStatusRecord", []):
                evse_id = evse.get("EvseID", "")
                evse_status = evse.get("EVSEStatus", "")

                records_status.append({
                    "Timestamp": timestamp,
                    "OperatorID": operator_id,
                    "OperatorName": operator_name,
                    "EvseID": evse_id,
                    "EVSEStatus": evse_status
                })

df = pd.DataFrame(records_status)
df["Timestamp"] = pd.to_datetime(df["Timestamp"], utc=True)
df

,Timestamp,OperatorID,OperatorName,EvseID,EVSEStatus
0,2025-10-23 09:22:48+00:00,CH*SWISSCHARGE,Swisscharge,CH*SWI*E10382,OutOfService
1,2025-10-23 09:22:48+00:00,CH*SWISSCHARGE,Swisscharge,CH*SWI*E10383,OutOfService
2,2025-10-23 09:22:48+00:00,CH*SWISSCHARGE,Swisscharge,CH*SWI*E262873,OutOfService
3,2025-10-23 09:22:48+00:00,CH*SWISSCHARGE,Swisscharge,CH*SWI*E262874,OutOfService
4,2025-10-23 09:22:48+00:00,CH*SWISSCHARGE,Swisscharge,CH*SWI*E262875,OutOfService
...,...,...,...,...,...
233277,2025-10-23 12:15:14+00:00,CH*LDL,Lidl Schweiz AG,CH*LDL*E00000014,Available
233278,2025-10-23 12:15:14+00:00,CH*LDL,Lidl Schweiz AG,CH*LDL*E00000448,Available
233279,2025-10-23 12:15:14+00:00,CH*LDL,Lidl Schweiz AG,CH*LDL*E00000130,Available
233280,2025-10-23 12:15:14+00:00,CH*LDL,Lidl Schweiz AG,CH*LDL*E00000131,Available
